In [ ]:
import numpy as np
from prettytable import PrettyTable
from collections import defaultdict


def read_log_file(log_path, print_pretty_table=False, append_all=False):
    data = defaultdict(list)
    with open(log_path, 'r') as file:
        file.readline()
        for line in file.readlines():
            line = line.strip().split(',')
            video, user, trace = list(map(int, line[:3]))
            qoe_w1, qoe_w2, qoe_w3, qoe, qoe1, qoe2, qoe3 = list(map(float, line[3:]))
            qoe_w1, qoe_w2, qoe_w3 = int(qoe_w1), int(qoe_w2), int(qoe_w3)
            data[(qoe_w1, qoe_w2, qoe_w3)].append([video, user, trace, qoe, qoe1, qoe2, qoe3])
            if append_all:
                data[(-1, -1, -1)].append([video, user, trace, qoe, qoe1, qoe2, qoe3])

    if print_pretty_table:
        for qoe_weight, values in data.items():
            print('On QoE Weight:', qoe_weight)
            pt = PrettyTable()
            pt.field_names = ['video', 'user', 'trace', 'qoe', 'qoe1', 'qoe2', 'qoe3']
            mean_qoe, mean_qoe1, mean_qoe2, mean_qoe3 = 0., 0., 0., 0.
            for value in values:
                pt.add_row(value)
                mean_qoe += value[3]
                mean_qoe1 += value[4]
                mean_qoe2 += value[5]
                mean_qoe3 += value[6]
            mean_qoe /= len(values)
            mean_qoe1 /= len(values)
            mean_qoe2 /= len(values)
            mean_qoe3 /= len(values)
            pt.add_row([-1, -1, -1, mean_qoe, mean_qoe1, mean_qoe2, mean_qoe3])
            print(pt)

    return data


def organize_data(data: dict, print_organized_data=False):
    organized_data = {}
    for qoe_weight, values in data.items():
        organized_data[qoe_weight] = {'qoe': [], 'qoe1': [], 'qoe2': [], 'qoe3': []}
        for value in values:
            organized_data[qoe_weight]['qoe'].append(value[3])
            organized_data[qoe_weight]['qoe1'].append(value[4])
            organized_data[qoe_weight]['qoe2'].append(value[5])
            organized_data[qoe_weight]['qoe3'].append(value[6])
        for key, value in organized_data[qoe_weight].items():
            organized_data[qoe_weight][key] = np.array(value)
    
    if print_organized_data:
        for qoe_weight, values in organized_data.items():
            print('On QoE Weight:', qoe_weight)
            for key, value in values.items():
                print(key, ':', value.tolist(), '\n')
    
    return organized_data

In [ ]:
def append_all(data):
    total_qoe, total_qoe1, total_qoe2, total_qoe3 = [], [], [], []
    for key, value in data.items():
        total_qoe.extend(value['qoe'].tolist())
        total_qoe1.extend(value['qoe1'].tolist())
        total_qoe2.extend(value['qoe2'].tolist())
        total_qoe3.extend(value['qoe3'].tolist())
    total_qoe = np.array(total_qoe)
    total_qoe1 = np.array(total_qoe1)
    total_qoe2 = np.array(total_qoe2)
    total_qoe3 = np.array(total_qoe3)
    data[(-1, -1, -1)] = {'qoe': total_qoe, 'qoe1': total_qoe1, 'qoe2': total_qoe2, 'qoe3': total_qoe3}
    return data

# Our Model vs. Baselines

In [ ]:
# rlva data
# Wu2017
rlva_seen_fpaths = [
    '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/rlva/Wu2017_4G/seen_qoe0/epochs_100_bs_256_lr_0.0001_gamma_0.99_seed_1_ent_0.1_results.csv',  # qoe 0
    '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/rlva/Wu2017_4G/seen_qoe1/epochs_100_bs_256_lr_0.0001_gamma_0.99_seed_2_ent_0.1_results.csv',  # qoe 1
    '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/rlva/Wu2017_4G/seen_qoe2/epochs_100_bs_256_lr_0.0001_gamma_0.99_seed_1_ent_0.1_results.csv',  # qoe 2
    '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/rlva/Wu2017_4G/seen_qoe3/epochs_100_bs_256_lr_0.0001_gamma_0.99_seed_1_ent_0.1_results.csv'
]
rlva_unseen_fpaths = [
    '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/rlva/Wu2017_4G/unseen_qoe0/epochs_100_bs_256_lr_0.0001_gamma_0.99_seed_1_ent_0.1_results.csv',  # qoe 0
    '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/rlva/Wu2017_4G/unseen_qoe1/epochs_100_bs_256_lr_0.0001_gamma_0.99_seed_2_ent_0.1_results.csv',  # qoe 1
    '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/rlva/Wu2017_4G/unseen_qoe2/epochs_100_bs_256_lr_0.0001_gamma_0.99_seed_1_ent_0.1_results.csv',  # qoe 2
    '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/rlva/Wu2017_4G/unseen_qoe3/epochs_100_bs_256_lr_0.0001_gamma_0.99_seed_1_ent_0.1_results.csv',  # qoe 3
]
# Jin2022
# rlva_seen_fpaths = [
#     # '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/rlva/Jin2022_4G/seen_qoe0/epochs_200_bs_256_lr_0.0001_gamma_0.99_seed_1_ent_0.1_results.csv',  # qoe 0
#     '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/rlva/Jin2022_4G/seen_qoe0/epochs_100_bs_256_lr_0.0001_gamma_0.99_seed_1_ent_0.01_results.csv',
#     # '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/rlva/Jin2022_4G/seen_qoe1/epochs_200_bs_256_lr_0.0001_gamma_0.99_seed_1_ent_0.1_results.csv',  # qoe 1
#     # '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/rlva/Jin2022_4G/seen_qoe1/epochs_100_bs_256_lr_0.0001_gamma_0.99_seed_1_ent_0.1_results.csv',  # qoe 1
#     '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/rlva/Jin2022_4G/seen_qoe1/epochs_100_bs_256_lr_0.0001_gamma_0.99_seed_1_ent_0.01_results.csv',
#     '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/rlva/Jin2022_4G/seen_qoe2/epochs_100_bs_256_lr_0.0001_gamma_0.99_seed_1_ent_0.1_results.csv',  # qoe 2
#     # '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/rlva/Jin2022_4G/seen_qoe3/epochs_200_bs_256_lr_0.0001_gamma_0.99_seed_1_ent_0.1_results.csv'
#     # '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/rlva/Jin2022_4G/seen_qoe3/epochs_100_bs_256_lr_0.0001_gamma_0.99_seed_1_ent_0.1_results.csv'
#     '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/rlva/Jin2022_4G/seen_qoe3/epochs_100_bs_256_lr_0.0001_gamma_0.99_seed_1_ent_0.05_results.csv',
# ]
# rlva_unseen_fpaths = [
#     # '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/rlva/Jin2022_4G/unseen_qoe0/epochs_200_bs_256_lr_0.0001_gamma_0.99_seed_1_ent_0.1_results.csv',  # qoe 0
#     '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/rlva/Jin2022_4G/unseen_qoe0/epochs_100_bs_256_lr_0.0001_gamma_0.99_seed_1_ent_0.01_results.csv',
#     # '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/rlva/Jin2022_4G/unseen_qoe1/epochs_200_bs_256_lr_0.0001_gamma_0.99_seed_1_ent_0.1_results.csv',  # qoe 1
#     '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/rlva/Jin2022_4G/unseen_qoe1/epochs_100_bs_256_lr_0.0001_gamma_0.99_seed_1_ent_0.01_results.csv',
#     # '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/rlva/Jin2022_4G/unseen_qoe1/epochs_100_bs_256_lr_0.0001_gamma_0.99_seed_1_ent_0.1_results.csv',  # qoe 1
#     '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/rlva/Jin2022_4G/unseen_qoe2/epochs_100_bs_256_lr_0.0001_gamma_0.99_seed_1_ent_0.1_results.csv',  # qoe 2
#     # '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/rlva/Jin2022_4G/unseen_qoe3/epochs_200_bs_256_lr_0.0001_gamma_0.99_seed_1_ent_0.1_results.csv',  # qoe 3
#     # '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/rlva/Jin2022_4G/unseen_qoe3/epochs_100_bs_256_lr_0.0001_gamma_0.99_seed_1_ent_0.1_results.csv',  # qoe 3
#     '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/rlva/Jin2022_4G/unseen_qoe3/epochs_100_bs_256_lr_0.0001_gamma_0.99_seed_1_ent_0.05_results.csv'
# ]
rlva_seen = {}
for rlva_seen_fpath in rlva_seen_fpaths:
    rlva_seen.update(organize_data(read_log_file(rlva_seen_fpath, append_all=False, print_pretty_table=False)))
rlva_unseen = {}
for rlva_unseen_fpath in rlva_unseen_fpaths:
    rlva_unseen.update(organize_data(read_log_file(rlva_unseen_fpath, append_all=False, print_pretty_table=False)))

In [ ]:
# paas data
# Wu2017
paas_seen_fpaths = [
    # '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/paas/Wu2017_4G/seen_qoe0_1_2_3/epochs_100_bs_256_lr_0.0001_gamma_0.99_seed_1_tuf_500_results.csv',
    # '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/paas2/Wu2017_4G/seen_qoe0_1_2_3/epochs_20_bs_256_lr_0.0001_gamma_0.95_seed_15_tuf_50_results.csv',
    # '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/paas2/Wu2017_4G/seen_qoe0_1_2_3/epochs_22_bs_256_lr_0.0001_gamma_0.95_seed_15_tuf_50_results.csv',
    '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/paas2/Wu2017_4G/seen_qoe0_1_2_3/epochs_500_bs_256_lr_0.0001_gamma_0.95_seed_15_tuf_50_results.csv'
]
paas_unseen_fpaths = [
    # '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/paas/Wu2017_4G/unseen_qoe0_1_2_3/epochs_100_bs_256_lr_0.0001_gamma_0.99_seed_1_tuf_500_results.csv',
    # '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/paas2/Wu2017_4G/unseen_qoe0_1_2_3/epochs_20_bs_256_lr_0.0001_gamma_0.95_seed_15_tuf_50_results.csv',
    # '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/paas2/Wu2017_4G/unseen_qoe0_1_2_3/epochs_22_bs_256_lr_0.0001_gamma_0.95_seed_15_tuf_50_results.csv',
    '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/paas2/Wu2017_4G/unseen_qoe0_1_2_3/epochs_500_bs_256_lr_0.0001_gamma_0.95_seed_15_tuf_50_results.csv'
]
# Jin2022
# paas_seen_fpaths = [
#     '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/paas2/Jin2022_4G/seen_qoe0_1_2_3/epochs_100_bs_256_lr_0.0001_gamma_0.95_seed_15_tuf_50_results.csv'
# ]
# paas_unseen_fpaths = [
#     '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/paas2/Jin2022_4G/unseen_qoe0_1_2_3/epochs_100_bs_256_lr_0.0001_gamma_0.95_seed_15_tuf_50_results.csv'
# ]
paas_seen = {}
for paas_seen_fpath in paas_seen_fpaths:
    paas_seen.update(organize_data(read_log_file(paas_seen_fpath, append_all=False, print_pretty_table=False)))
paas_unseen = {}
for paas_unseen_fpath in paas_unseen_fpaths:
    paas_unseen.update(organize_data(read_log_file(paas_unseen_fpath, append_all=False, print_pretty_table=False)))


In [ ]:
# parima data
# Wu2017
parima_seen_fpaths = [
    # '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/parima/Wu2017_4G/seen_qoe0_1_2_3/results.csv',
    '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/parima/Wu2017_4G/seen_qoe0/seed_1_agg_3.0/results.csv',
    # '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/parima/Wu2017_4G/seen_qoe1/seed_1_agg_0.5/results.csv',
    '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/parima/Wu2017_4G/seen_qoe1/seed_1_agg_0.8/results.csv',
    '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/parima/Wu2017_4G/seen_qoe2/seed_1_agg_1.0/results.csv',
    '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/parima/Wu2017_4G/seen_qoe3/seed_1_agg_0.8/results.csv'
]
parima_unseen_fpaths = [
    # '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/parima/Wu2017_4G/unseen_qoe0_1_2_3/results.csv',
    '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/parima/Wu2017_4G/unseen_qoe0/seed_1_agg_3.0/results.csv',
    '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/parima/Wu2017_4G/unseen_qoe1/seed_1_agg_0.5/results.csv',
    # '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/parima/Wu2017_4G/unseen_qoe1/seed_1_agg_0.6/results.csv',
    '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/parima/Wu2017_4G/unseen_qoe2/seed_1_agg_1.0/results.csv',
    '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/parima/Wu2017_4G/unseen_qoe3/seed_1_agg_0.8/results.csv'
]
# Jin2022
# parima_seen_fpaths = [
#     # '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/parima/Jin2022_4G/seen_qoe0/seed_1_agg_8.0/results.csv',
#     '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/parima/Jin2022_4G/seen_qoe0/seed_1_agg_5.0/results.csv',
#     '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/parima/Jin2022_4G/seen_qoe1/seed_1_agg_0.5/results.csv',
#     # '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/parima/Jin2022_4G/seen_qoe2/seed_1_agg_1.0/results.csv',
#     '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/parima/Jin2022_4G/seen_qoe2/seed_1_agg_2.0/results.csv',
#     # '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/parima/Jin2022_4G/seen_qoe3/seed_1_agg_1.0/results.csv'
#     '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/parima/Jin2022_4G/seen_qoe3/seed_1_agg_2.0/results.csv'
# ]
# parima_unseen_fpaths = [
#     # '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/parima/Jin2022_4G/unseen_qoe0/seed_1_agg_8.0/results.csv',
#     '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/parima/Jin2022_4G/unseen_qoe0/seed_1_agg_5.0/results.csv',
#     '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/parima/Jin2022_4G/unseen_qoe1/seed_1_agg_0.5/results.csv',
#     # '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/parima/Jin2022_4G/unseen_qoe2/seed_1_agg_1.0/results.csv',
#     '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/parima/Jin2022_4G/unseen_qoe2/seed_1_agg_2.0/results.csv',
#     '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/parima/Jin2022_4G/unseen_qoe3/seed_1_agg_1.0/results.csv',
# ]
parima_seen = {}
for parima_seen_fpath in parima_seen_fpaths:
    parima_seen.update(organize_data(read_log_file(parima_seen_fpath, append_all=False, print_pretty_table=False)))
parima_unseen = {}
for parima_unseen_fpath in parima_unseen_fpaths:
    parima_unseen.update(organize_data(read_log_file(parima_unseen_fpath, append_all=False, print_pretty_table=False)))

In [ ]:
# gen360 data
# Wu2017
gen360_seen_fpaths = [
    '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/gen360_3/Wu2017_4G/seen_qoe0_1_2_3/epochs_1000_bs_512_lr_0.0005_gamma_0.95_seed_666_ent_0.02_useid_True_lambda_0.5_ilr_0.0001_iur_2_bc_True/results.csv'
]
gen360_unseen_fpaths = [
    # '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/gen360/Wu2017_4G/unseen_qoe0_1_2_3/epochs_2000_bs_512_lr_0.0001_gamma_0.95_seed_1_ent_0.02_useid_True_lambda_0.5_ilr_0.0001_iur_1_bc_True/results.csv'
    '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/gen360_3/Wu2017_4G/unseen_qoe0_1_2_3/epochs_1000_bs_512_lr_0.0005_gamma_0.95_seed_666_ent_0.02_useid_True_lambda_0.5_ilr_0.0001_iur_2_bc_True/results.csv'
]
# Jin2022
# gen360_seen_fpaths = [
#     # '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/gen360_3/Jin2022_4G/seen_qoe0_1_2_3/epochs_1000_bs_512_lr_0.0005_gamma_0.95_seed_5_ent_0.02_useid_True_lambda_0.5_ilr_0.0001_iur_2_bc_True/results.csv',
#     '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/gen360_3/Jin2022_4G/seen_qoe0_1_2_3/epochs_1000_bs_512_lr_0.0005_gamma_0.95_seed_5_ent_0.02_useid_True_lambda_0.5_ilr_0.0001_iur_2_bc_False/results.csv'
# ]
# gen360_unseen_fpaths = [
#     # '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/gen360_3/Jin2022_4G/unseen_qoe0_1_2_3/epochs_1000_bs_512_lr_0.0005_gamma_0.95_seed_5_ent_0.02_useid_True_lambda_0.5_ilr_0.0001_iur_2_bc_True/results.csv',
#     '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/gen360_3/Jin2022_4G/unseen_qoe0_1_2_3/epochs_1000_bs_512_lr_0.0005_gamma_0.95_seed_5_ent_0.02_useid_True_lambda_0.5_ilr_0.0001_iur_2_bc_False/results.csv'
# ]
gen360_seen = {}
for gen360_seen_fpath in gen360_seen_fpaths:
    gen360_seen.update(organize_data(read_log_file(gen360_seen_fpath, append_all=False, print_pretty_table=False)))
gen360_unseen = {}
for gen360_unseen_fpath in gen360_unseen_fpaths:
    gen360_unseen.update(organize_data(read_log_file(gen360_unseen_fpath, append_all=False, print_pretty_table=False)))

In [ ]:
rlva_seen = append_all(rlva_seen)
rlva_unseen = append_all(rlva_unseen)

In [ ]:
paas_seen = append_all(paas_seen)
paas_unseen = append_all(paas_unseen)

In [ ]:
parima_seen = append_all(parima_seen)
parima_unseen = append_all(parima_unseen)

In [ ]:
gen360_seen = append_all(gen360_seen)
gen360_unseen = append_all(gen360_unseen)

In [ ]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt


def plot_box_overall_qoe(parima_1, rlva_1, paas_1, gen360_1, parima_2, rlva_2, paas_2, gen360_2):
    matplotlib.rcParams['hatch.linewidth'] = 1
    data_1 = []
    data_2 = []
    data_1.append(parima_1[-1, -1, -1]['qoe'])
    data_1.append(rlva_1[-1, -1, -1]['qoe'])
    # print(rlva_1[-1, -1, -1]['qoe'])
    data_1.append(paas_1[-1, -1, -1]['qoe'])
    data_1.append(gen360_1[-1, -1, -1]['qoe'])

    data_2.append(parima_2[-1, -1, -1]['qoe'])
    data_2.append(rlva_2[-1, -1, -1]['qoe'])
    data_2.append(paas_2[-1, -1, -1]['qoe'])
    data_2.append(gen360_2[-1, -1, -1]['qoe'])

    # search min, max for normalization
    min_1, max_1 = np.min(data_1), np.max(data_1)
    min_2, max_2 = np.min(data_2), np.max(data_2)

    # normalize
    for i in range(len(data_1)):
        data_1[i] = (data_1[i] - min_1) / (max_1 - min_1)
        data_2[i] = (data_2[i] - min_2) / (max_2 - min_2)

    plt.figure(figsize=(7, 5))

    # colors = ['pink', 'lightblue', 'lightgreen', '#D38C80']
    colors = ['#F8B5C1', '#9FD18C', '#B0D9E7', '#D5A3CB']
    xlabels = ['', '              On Trained QoE Preferences', '', '']
    hatchs = ['//', '\\\\', '+', 'x']
    widths = [0.6] * 4
    meanprops = dict(marker='^', markersize=8, markeredgecolor='black', markerfacecolor='orange', linewidth=0.0)
    positions = np.arange(1, 5)
    print(positions)

    boxes = plt.boxplot(data_1, notch=False, vert=True, patch_artist=True, showfliers=False, showmeans=True,
                        positions=positions, labels=xlabels, widths=widths, meanprops=meanprops)
    index = 0
    for box in boxes['boxes']:
        box.set(color='black', linewidth=1)  # 箱体边框颜色
        box.set(facecolor=colors[index], alpha=1)  # 箱体内部填充颜色
        box.set(hatch=hatchs[index])
        index += 1

    colors = ['#F8B5C1', '#9FD18C', '#B0D9E7', '#D5A3CB']
    xlabels = ['', '              On Unseen QoE Preferences', '', '']
    hatchs = ['//', '\\\\', '+', 'x']
    boxes = plt.boxplot(data_2, notch=False, vert=True, patch_artist=True, labels=xlabels, showfliers=False,
                      showmeans=True, positions=positions + 5, widths=widths, meanprops=meanprops)

    index = 0
    for box in boxes['boxes']:
        box.set(color='black', linewidth=1)  # 箱体边框颜色
        box.set(facecolor=colors[index], alpha=1)  # 箱体内部填充颜色
        box.set(hatch=hatchs[index])
        index += 1

    plt.ylim([0.4, 1.08])
    # plt.ylim([-0.7, 0.7])
    # plt.xticks([1, 2, 3, 4, 6, 7, 8, 9], labels)
    # plt.xticks([2.5,  7.5], labels)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.ylabel('Normalized Mean QoE', fontsize=14)
    # plt.title('Average QoE')
    plt.grid(axis='y', linestyle='--', alpha=0.5)
    # plt.grid(axis='x', linestyle='--', alpha=0.5)
    plt.tight_layout()
    labels = ['PARIMA', 'RLVA', 'PAAS', 'GEN360']
    plt.legend(boxes['boxes'], labels, fontsize=13, ncol=4, loc='upper center')
    # plt.legend(boxes['boxes'], labels, fontsize=12, ncol=4, loc='upper center')
    # plt.savefig("overall_qoe_jin2022.pdf", dpi=300, format="pdf")
    plt.savefig("overall_qoe_wu2017.pdf", dpi=300, format="pdf")
    plt.show()


plot_box_overall_qoe(parima_seen, rlva_seen, paas_seen, gen360_seen, parima_unseen, rlva_unseen, paas_unseen, gen360_unseen)

In [ ]:
def plot_box_individual_qoe(parima, rlva, paas, gen360, qoe_weight):
    data_qoe = []
    data_qoe1 = []
    data_qoe2 = []
    data_qoe3 = []

    data_qoe.append(parima[qoe_weight]['qoe'])
    data_qoe.append(rlva[qoe_weight]['qoe'])
    data_qoe.append(paas[qoe_weight]['qoe'])
    data_qoe.append(gen360[qoe_weight]['qoe'])

    data_qoe1.append(parima[qoe_weight]['qoe1'])
    data_qoe1.append(rlva[qoe_weight]['qoe1'])
    data_qoe1.append(paas[qoe_weight]['qoe1'])
    data_qoe1.append(gen360[qoe_weight]['qoe1'])

    data_qoe2.append(parima[qoe_weight]['qoe2'])
    data_qoe2.append(rlva[qoe_weight]['qoe2'])
    data_qoe2.append(paas[qoe_weight]['qoe2'])
    data_qoe2.append(gen360[qoe_weight]['qoe2'])

    data_qoe3.append(parima[qoe_weight]['qoe3'])
    data_qoe3.append(rlva[qoe_weight]['qoe3'])
    data_qoe3.append(paas[qoe_weight]['qoe3'])
    data_qoe3.append(gen360[qoe_weight]['qoe3'])

    # search min, max for normalization
    min_, max_ = np.min(data_qoe), np.max(data_qoe)
    min_1, max_1 = np.min(data_qoe1), np.max(data_qoe1)
    min_2, max_2 = np.min(data_qoe2), np.max(data_qoe2)
    min_3, max_3 = np.min(data_qoe3), np.max(data_qoe3)

    # normalize
    # for i in range(len(data_qoe1)):
    #     data_qoe[i] = (data_qoe[i] - min_) / (max_ - min_)
    #     data_qoe1[i] = (data_qoe1[i] - min_1) / (max_1 - min_1)
    #     data_qoe2[i] = (data_qoe2[i] - min_2) / (max_2 - min_2)
    #     data_qoe3[i] = (data_qoe3[i] - min_3) / (max_3 - min_3)

    plt.figure(figsize=(8, 5))

    # colors = ['pink', 'lightblue', 'lightgreen', '#D38C80']
    colors = ['#F8B5C1', '#9FD18C', '#B0D9E7', '#D5A3CB']
    xlabels = ['', '   Overall QoE', '', '']
    hatchs = ['//', '\\\\', '+', 'x']
    widths = [0.5] * 4
    meanprops = dict(marker='^', markersize=8, markeredgecolor='black', markerfacecolor='orange', linewidth=0.0)
    positions = np.arange(1, 5)
    print(positions)

    boxes = plt.boxplot(data_qoe, notch=False, vert=True, patch_artist=True, showfliers=False, showmeans=True,
                        positions=positions, labels=xlabels, widths=widths, meanprops=meanprops)
    index = 0
    for box in boxes['boxes']:
        box.set(color='black', linewidth=1)  # 箱体边框颜色
        box.set(facecolor=colors[index], alpha=1)  # 箱体内部填充颜色
        box.set(hatch=hatchs[index])
        index += 1

    # colors = ['pink', 'lightblue', 'lightgreen', '#D38C80']
    colors = ['#F8B5C1', '#9FD18C', '#B0D9E7', '#D5A3CB']
    xlabels = ['', '   Viewport Quality', '', '']
    hatchs = ['//', '\\\\', '+', 'x']
    widths = [0.5] * 4
    meanprops = dict(marker='^', markersize=8, markeredgecolor='black', markerfacecolor='orange', linewidth=0.0)
    positions = np.arange(1, 5)
    print(positions)

    boxes = plt.boxplot(data_qoe1, notch=False, vert=True, patch_artist=True, showfliers=False, showmeans=True,
                        positions=positions + 5 * 1, labels=xlabels, widths=widths, meanprops=meanprops)
    index = 0
    for box in boxes['boxes']:
        box.set(color='black', linewidth=1)  # 箱体边框颜色
        box.set(facecolor=colors[index], alpha=1)  # 箱体内部填充颜色
        box.set(hatch=hatchs[index])
        index += 1

    colors = ['#F8B5C1', '#9FD18C', '#B0D9E7', '#D5A3CB']
    xlabels = ['', '    Rebuffering', '', '']
    hatchs = ['//', '\\\\', '+', 'x']
    boxes = plt.boxplot(data_qoe2, notch=False, vert=True, patch_artist=True, labels=xlabels, showfliers=False,
                      showmeans=True, positions=positions + 5 * 2, widths=widths, meanprops=meanprops)

    index = 0
    for box in boxes['boxes']:
        box.set(color='black', linewidth=1)  # 箱体边框颜色
        box.set(facecolor=colors[index], alpha=1)  # 箱体内部填充颜色
        box.set(hatch=hatchs[index])
        index += 1

    colors = ['#F8B5C1', '#9FD18C', '#B0D9E7', '#D5A3CB']
    xlabels = ['', '    Quality Smoothness', '', '']
    hatchs = ['//', '\\\\', '+', 'x']
    boxes = plt.boxplot(data_qoe3, notch=False, vert=True, patch_artist=True, labels=xlabels, showfliers=False,
                      showmeans=True, positions=positions + 5 * 3, widths=widths, meanprops=meanprops)

    index = 0
    for box in boxes['boxes']:
        box.set(color='black', linewidth=1)  # 箱体边框颜色
        box.set(facecolor=colors[index], alpha=1)  # 箱体内部填充颜色
        box.set(hatch=hatchs[index])
        index += 1

    # plt.ylim([0, 1])
    # plt.xticks([1, 2, 3, 4, 6, 7, 8, 9], labels)
    # plt.xticks([2.5,  7.5], labels)
    plt.xticks(fontsize=11.5)
    plt.yticks(fontsize=12)
    plt.ylabel('Normalized Mean Scores', fontsize=14)
    plt.title(f'QoE Weight {qoe_weight}')
    plt.grid(axis='y', linestyle='--', alpha=0.5)
    # plt.grid(axis='x', linestyle='--', alpha=0.5)
    plt.tight_layout()
    labels = ['PARIMA', 'RLVA', 'PAAS', 'GEN360']
    plt.legend(boxes['boxes'], labels, fontsize=14, bbox_to_anchor=(0.55, 1),)
    plt.savefig(f"individual_qoe_jin2022{'_'.join(map(str, qoe_weight))}.pdf", dpi=300, format="pdf")
    plt.show()

plot_box_individual_qoe(parima_seen, rlva_seen, paas_seen, gen360_seen, (7, 1, 1))
plot_box_individual_qoe(parima_seen, rlva_seen, paas_seen, gen360_seen, (1, 7, 1))
plot_box_individual_qoe(parima_seen, rlva_seen, paas_seen, gen360_seen, (1, 1, 7))
plot_box_individual_qoe(parima_seen, rlva_seen, paas_seen, gen360_seen, (3, 3, 3))

plot_box_individual_qoe(parima_unseen, rlva_unseen, paas_unseen, gen360_unseen, (5, 1, 3))
plot_box_individual_qoe(parima_unseen, rlva_unseen, paas_unseen, gen360_unseen, (2, 4, 3))
plot_box_individual_qoe(parima_unseen, rlva_unseen, paas_unseen, gen360_unseen, (1, 3, 5))
plot_box_individual_qoe(parima_unseen, rlva_unseen, paas_unseen, gen360_unseen, (4, 4, 1))

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

def plot_bar_individual_qoe(parima, rlva, paas, gen360, qoe_weight, title=None):
    data_qoe = []
    data_qoe1 = []
    data_qoe2 = []
    data_qoe3 = []

    data_qoe.append(parima[qoe_weight]['qoe'])
    data_qoe.append(rlva[qoe_weight]['qoe'])
    data_qoe.append(paas[qoe_weight]['qoe'])
    data_qoe.append(gen360[qoe_weight]['qoe'])

    data_qoe1.append(parima[qoe_weight]['qoe1'])
    data_qoe1.append(rlva[qoe_weight]['qoe1'])
    data_qoe1.append(paas[qoe_weight]['qoe1'])
    data_qoe1.append(gen360[qoe_weight]['qoe1'])

    data_qoe2.append(parima[qoe_weight]['qoe2'])
    data_qoe2.append(rlva[qoe_weight]['qoe2'])
    data_qoe2.append(paas[qoe_weight]['qoe2'])
    data_qoe2.append(gen360[qoe_weight]['qoe2'])

    data_qoe3.append(parima[qoe_weight]['qoe3'])
    data_qoe3.append(rlva[qoe_weight]['qoe3'])
    data_qoe3.append(paas[qoe_weight]['qoe3'])
    data_qoe3.append(gen360[qoe_weight]['qoe3'])

    # search min, max for normalization
    min_, max_ = np.min(data_qoe), np.max(data_qoe)
    min_1, max_1 = np.min(data_qoe1), np.max(data_qoe1)
    min_2, max_2 = np.min(data_qoe2), np.max(data_qoe2)
    min_3, max_3 = np.min(data_qoe3), np.max(data_qoe3)

    # normalize
    for i in range(len(data_qoe1)):
        data_qoe[i] = (data_qoe[i] - min_) / (max_ - min_)
        data_qoe1[i] = (data_qoe1[i] - min_1) / (max_1 - min_1)
        data_qoe2[i] = (data_qoe2[i] - min_2) / (max_2 - min_2)
        data_qoe3[i] = (data_qoe3[i] - min_3) / (max_3 - min_3)
    matplotlib.rcParams['hatch.linewidth'] = 2.5
    # fig, ax = plt.subplots(figsize=(8, 5))
    fig, ax = plt.subplots(figsize=(6, 3))
    width = 1
    qoe_num = 4
    method_num = 4
    x = []
    for i in range(qoe_num):
        for j in range(i + method_num * i, i + method_num * (i + 1)):
            x.append(j)
    print(x)
    total_data = []
    total_data_std = []
    tmp_data = [data_qoe, data_qoe1, data_qoe2, data_qoe3]
    for i in range(len(tmp_data)):
        for j in range(len(tmp_data[i])):
            total_data.append(np.mean(tmp_data[i][j]))
            total_data_std.append(np.std(tmp_data[i][j]))
            # total_data.append(np.mean(data_qoe1[i]))
            # total_data.append(np.mean(data_qoe2[i]))
            # total_data.append(np.mean(data_qoe3[i]))
    print(total_data)
    print(total_data_std)

    # data = [82.33797, 75.22432, 84.77188, 80.07726]
    colors = ['#6DBEC5', '#B9BB37', '#6DBEC5', '#B9BB37'] * method_num
    # colors = ['#41644A', '#2A3990', '#41644A', '#2A3990']
    colors = ['#F8B5C1', '#9FD18C', '#B0D9E7', '#D5A3CB'] * method_num
    hatchs = ['///', '\\\\\\', '++', 'xxx'] * method_num
    # hatchs = ['////', '\\\\\\\\', '+++', 'xxx'] * method_num
    labels = ['PARIMA', 'RLVA', 'PAAS', 'GEN360', 
              '', '', '', '', '', '', '', '', '', '', '', '',]
    for i in range(len(x)):
        ax.bar(x[i] - 0.5 * width, total_data[i], width, color='white', edgecolor=colors[i],
                        hatch=hatchs[i], zorder=100, lw=1., label=labels[i])
        # ax.bar(x[i] - 0.5 * width, total_data[i], width, color=colors[i], zorder=100, lw=1., edgecolor='black', label=labels[i])
        # ax.bar(x[i] - 0.5 * width, total_data[i], width, color='none', edgecolor='white',
        #                 hatch=hatchs[i], zorder=200, lw=1.)
        ax.bar(x[i] - 0.5 * width, total_data[i], width, color='none', edgecolor='black', zorder=200, lw=1., )
        # ax.errorbar(x[i] - 0.5 * width, total_data[i], yerr=total_data_std[i], color='black', lw=1., capsize=4, zorder=200)
        # ax.text(x[i] - 0.85 * width, data[i] + 1, '%.2f' % data[i], fontsize=12)
        # ax.text(x[i] - 0.85 * width, total_data[i] + 1, '%.1f' % total_data[i], fontsize=12)
    # xlabels = ['', '          On Trained Viewing Patterns', '', '', '',
    #            '', '            On Unseen Viewing Patterns', '', '', '']
    xlabels = ['              Overall QoE$\\uparrow$', '', '', '',
               '             Viewport\n              Quality$\\uparrow$', '', '', '', 
               '              Rebuffering$\\downarrow$', '', '', '', 
               '            Smoothness\n             Penalty$\\downarrow$', '', '', '',]
    # xlabels = ['        Overall QoE', '', '', '',
    #            '       Viewport Quality', '', '', '', 
    #            '       Rebuffering Time', '', '', '', 
    #            '       Smoothness Penalty', '', '', '',]
    ax.set_xticks(x)
    ax.set_xticklabels(xlabels, rotation=5)  # 14
    ax.set_ylim([0, 1])
    # ax.set_yticks([14, 15, 16, 17, 18])
    ax.set_ylabel('Normalized Mean Scores', fontsize=12)
    # ax.set_xlabel(' ', fontsize=14)
    # ax.legend(loc='upper right', fontsize=14, ncol=2, borderaxespad=0)  # 14
    ax.legend(loc='upper right', fontsize=10, ncol=2, borderaxespad=0)  # 14
    ax.grid(axis='y', alpha=0.6, zorder=10)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    if title is None:
        title = f'QoE Preference {qoe_weight}'
    plt.title(title, fontsize=12)
    plt.tight_layout()
    save_dir = ''
    # plt.savefig(f"individual_qoe_jin2022_{'_'.join(map(str, qoe_weight))}.pdf", dpi=300, format="pdf")
    plt.savefig(f"individual_qoe_{'_'.join(map(str, qoe_weight))}.pdf", dpi=300, format="pdf")
    # save_path = save_dir + 'motivation_accuracy_seen_unseen.pdf'
    # if save_path is not None:
    #     plt.savefig(save_path, dpi=600, bbox_inches='tight')
    plt.show()
    

plot_bar_individual_qoe(parima_seen, rlva_seen, paas_seen, gen360_seen, (7, 1, 1), title='QoE Preference ($\\frac{7}{9}$, $\\frac{1}{9}$, $\\frac{1}{9}$)')
plot_bar_individual_qoe(parima_seen, rlva_seen, paas_seen, gen360_seen, (1, 7, 1), title='QoE Preference ($\\frac{1}{9}$, $\\frac{7}{9}$, $\\frac{1}{9}$)')
plot_bar_individual_qoe(parima_seen, rlva_seen, paas_seen, gen360_seen, (1, 1, 7), title='QoE Preference ($\\frac{1}{9}$, $\\frac{1}{9}$, $\\frac{7}{9}$)')
plot_bar_individual_qoe(parima_seen, rlva_seen, paas_seen, gen360_seen, (3, 3, 3), title='QoE Preference ($\\frac{3}{9}$, $\\frac{3}{9}$, $\\frac{3}{9}$)')

plot_bar_individual_qoe(parima_unseen, rlva_unseen, paas_unseen, gen360_unseen, (5, 1, 3), title='QoE Preference ($\\frac{5}{9}$, $\\frac{1}{9}$, $\\frac{3}{9}$)')
plot_bar_individual_qoe(parima_unseen, rlva_unseen, paas_unseen, gen360_unseen, (2, 4, 3), title='QoE Preference ($\\frac{2}{9}$, $\\frac{4}{9}$, $\\frac{3}{9}$)')
plot_bar_individual_qoe(parima_unseen, rlva_unseen, paas_unseen, gen360_unseen, (1, 3, 5), title='QoE Preference ($\\frac{1}{9}$, $\\frac{3}{9}$, $\\frac{5}{9}$)')
plot_bar_individual_qoe(parima_unseen, rlva_unseen, paas_unseen, gen360_unseen, (4, 4, 1), title='QoE Preference ($\\frac{4}{9}$, $\\frac{4}{9}$, $\\frac{1}{9}$)')

# Ablation Study

In [ ]:
# gen360 data
# Wu2017
gen360_seen_fpaths = [
    '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/gen360_3/Wu2017_4G/seen_qoe0_1_2_3/epochs_1000_bs_512_lr_0.0005_gamma_0.95_seed_666_ent_0.02_useid_True_lambda_0.5_ilr_0.0001_iur_2_bc_True/results.csv'
]
gen360_unseen_fpaths = [
    # '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/gen360/Wu2017_4G/unseen_qoe0_1_2_3/epochs_2000_bs_512_lr_0.0001_gamma_0.95_seed_1_ent_0.02_useid_True_lambda_0.5_ilr_0.0001_iur_1_bc_True/results.csv'
    '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/gen360_3/Wu2017_4G/unseen_qoe0_1_2_3/epochs_1000_bs_512_lr_0.0005_gamma_0.95_seed_666_ent_0.02_useid_True_lambda_0.5_ilr_0.0001_iur_2_bc_True/results.csv'
]
# Jin2022
# gen360_seen_fpaths = [
#     # '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/gen360_3/Jin2022_4G/seen_qoe0_1_2_3/epochs_1000_bs_512_lr_0.0005_gamma_0.95_seed_5_ent_0.02_useid_True_lambda_0.5_ilr_0.0001_iur_2_bc_True/results.csv',
#     '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/gen360_3/Jin2022_4G/seen_qoe0_1_2_3/epochs_1000_bs_512_lr_0.0005_gamma_0.95_seed_5_ent_0.02_useid_True_lambda_0.5_ilr_0.0001_iur_2_bc_False/results.csv'
# ]
# gen360_unseen_fpaths = [
#     # '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/gen360_3/Jin2022_4G/unseen_qoe0_1_2_3/epochs_1000_bs_512_lr_0.0005_gamma_0.95_seed_5_ent_0.02_useid_True_lambda_0.5_ilr_0.0001_iur_2_bc_True/results.csv',
#     '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/gen360_3/Jin2022_4G/unseen_qoe0_1_2_3/epochs_1000_bs_512_lr_0.0005_gamma_0.95_seed_5_ent_0.02_useid_True_lambda_0.5_ilr_0.0001_iur_2_bc_False/results.csv'
# ]
gen360_seen = {}
for gen360_seen_fpath in gen360_seen_fpaths:
    gen360_seen.update(organize_data(read_log_file(gen360_seen_fpath, append_all=True, print_pretty_table=True)))
gen360_unseen = {}
for gen360_unseen_fpath in gen360_unseen_fpaths:
    gen360_unseen.update(organize_data(read_log_file(gen360_unseen_fpath, append_all=False, print_pretty_table=False)))

In [ ]:
# gen360 w/o RepL data
# Wu2017
gen360_wo_repl_seen_fpaths = [
    '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/gen360_3/Wu2017_4G/seen_qoe0_1_2_3/epochs_1000_bs_512_lr_0.0005_gamma_0.95_seed_5_ent_0.02_useid_False_lambda_0.5_ilr_0.0001_iur_2_bc_False/results.csv'
]
gen360_wo_repl_unseen_fpaths = [
    '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/gen360_3/Wu2017_4G/unseen_qoe0_1_2_3/epochs_1000_bs_512_lr_0.0005_gamma_0.95_seed_5_ent_0.02_useid_False_lambda_0.5_ilr_0.0001_iur_2_bc_False/results.csv'
]
# Jin2022
# gen360_seen_fpaths = [
#     # '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/gen360_3/Jin2022_4G/seen_qoe0_1_2_3/epochs_1000_bs_512_lr_0.0005_gamma_0.95_seed_5_ent_0.02_useid_True_lambda_0.5_ilr_0.0001_iur_2_bc_True/results.csv',
#     '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/gen360_3/Jin2022_4G/seen_qoe0_1_2_3/epochs_1000_bs_512_lr_0.0005_gamma_0.95_seed_5_ent_0.02_useid_True_lambda_0.5_ilr_0.0001_iur_2_bc_False/results.csv'
# ]
# gen360_unseen_fpaths = [
#     # '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/gen360_3/Jin2022_4G/unseen_qoe0_1_2_3/epochs_1000_bs_512_lr_0.0005_gamma_0.95_seed_5_ent_0.02_useid_True_lambda_0.5_ilr_0.0001_iur_2_bc_True/results.csv',
#     '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/gen360_3/Jin2022_4G/unseen_qoe0_1_2_3/epochs_1000_bs_512_lr_0.0005_gamma_0.95_seed_5_ent_0.02_useid_True_lambda_0.5_ilr_0.0001_iur_2_bc_False/results.csv'
# ]
gen360_wo_repl_seen = {}
for gen360_wo_repl_seen_fpath in gen360_wo_repl_seen_fpaths:
    gen360_wo_repl_seen.update(organize_data(read_log_file(gen360_wo_repl_seen_fpath, append_all=False, print_pretty_table=True)))
gen360_wo_repl_unseen = {}
for gen360_wo_repl_unseen_fpath in gen360_wo_repl_unseen_fpaths:
    gen360_wo_repl_unseen.update(organize_data(read_log_file(gen360_wo_repl_unseen_fpath, append_all=False, print_pretty_table=True)))

In [ ]:
# gen360 w/o BC data
# Wu2017
gen360_wo_bc_seen_fpaths = [
    '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/gen360_3/Wu2017_4G/seen_qoe0_1_2_3/epochs_1000_bs_512_lr_0.0005_gamma_0.95_seed_5_ent_0.02_useid_True_lambda_0.5_ilr_0.0001_iur_2_bc_False/results.csv'
]
gen360_wo_bc_unseen_fpaths = [
    '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/gen360_3/Wu2017_4G/unseen_qoe0_1_2_3/epochs_1000_bs_512_lr_0.0005_gamma_0.95_seed_5_ent_0.02_useid_True_lambda_0.5_ilr_0.0001_iur_2_bc_False/results.csv'
]
# Jin2022
# gen360_seen_fpaths = [
#     # '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/gen360_3/Jin2022_4G/seen_qoe0_1_2_3/epochs_1000_bs_512_lr_0.0005_gamma_0.95_seed_5_ent_0.02_useid_True_lambda_0.5_ilr_0.0001_iur_2_bc_True/results.csv',
#     '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/gen360_3/Jin2022_4G/seen_qoe0_1_2_3/epochs_1000_bs_512_lr_0.0005_gamma_0.95_seed_5_ent_0.02_useid_True_lambda_0.5_ilr_0.0001_iur_2_bc_False/results.csv'
# ]
# gen360_unseen_fpaths = [
#     # '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/gen360_3/Jin2022_4G/unseen_qoe0_1_2_3/epochs_1000_bs_512_lr_0.0005_gamma_0.95_seed_5_ent_0.02_useid_True_lambda_0.5_ilr_0.0001_iur_2_bc_True/results.csv',
#     '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/gen360_3/Jin2022_4G/unseen_qoe0_1_2_3/epochs_1000_bs_512_lr_0.0005_gamma_0.95_seed_5_ent_0.02_useid_True_lambda_0.5_ilr_0.0001_iur_2_bc_False/results.csv'
# ]
gen360_wo_bc_seen = {}
for gen360_wo_bc_seen_fpath in gen360_wo_bc_seen_fpaths:
    gen360_wo_bc_seen.update(organize_data(read_log_file(gen360_wo_bc_seen_fpath, append_all=False, print_pretty_table=True)))
gen360_wo_bc_unseen = {}
for gen360_wo_bc_unseen_fpath in gen360_wo_bc_unseen_fpaths:
    gen360_wo_bc_unseen.update(organize_data(read_log_file(gen360_wo_bc_unseen_fpath, append_all=False, print_pretty_table=True)))

In [ ]:
gen360_seen = append_all(gen360_seen)
gen360_unseen = append_all(gen360_unseen)

In [ ]:
gen360_wo_repl_seen = append_all(gen360_wo_repl_seen)
gen360_wo_repl_unseen = append_all(gen360_wo_repl_unseen)

In [ ]:
gen360_wo_bc_seen = append_all(gen360_wo_bc_seen)
gen360_wo_bc_unseen = append_all(gen360_wo_bc_unseen)

In [ ]:
print(gen360_seen[-1, -1, -1])

In [ ]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt


def plot_box_overall_qoe(gen360_1, gen360_wo_repl_1, gen360_wo_bc_1, gen360_2, gen360_wo_repl_2, gen360_wo_bc_2):
    matplotlib.rcParams['hatch.linewidth'] = 1
    data_1 = []
    data_2 = []
    data_1.append(gen360_1[-1, -1, -1]['qoe'])
    data_1.append(gen360_wo_repl_1[-1, -1, -1]['qoe'])
    # print(rlva_1[-1, -1, -1]['qoe'])
    data_1.append(gen360_wo_bc_1[-1, -1, -1]['qoe'])

    data_2.append(gen360_2[-1, -1, -1]['qoe'])
    data_2.append(gen360_wo_repl_2[-1, -1, -1]['qoe'])
    data_2.append(gen360_wo_bc_2[-1, -1, -1]['qoe'])

    # search min, max for normalization
    min_1, max_1 = np.min(data_1), np.max(data_1)
    min_2, max_2 = np.min(data_2), np.max(data_2)

    # normalize
    # for i in range(len(data_1)):
    #     data_1[i] = (data_1[i] - min_1) / (max_1 - min_1)
    #     data_2[i] = (data_2[i] - min_2) / (max_2 - min_2)

    plt.figure(figsize=(7, 5))

    # colors = ['pink', 'lightblue', 'lightgreen', '#D38C80']
    colors = ['#D5A3CB', '#F8B5C1', '#9FD18C', ]
    xlabels = ['', '              On Trained QoE Preferences', '']
    hatchs = ['x', '//', '\\\\']
    widths = [0.6] * 3
    meanprops = dict(marker='^', markersize=8, markeredgecolor='black', markerfacecolor='orange', linewidth=0.0)
    positions = np.arange(1, 4)
    print(positions)

    boxes = plt.boxplot(data_1, notch=False, vert=True, patch_artist=True, showfliers=False, showmeans=True,
                        positions=positions, labels=xlabels, widths=widths, meanprops=meanprops)
    index = 0
    for box in boxes['boxes']:
        box.set(color='black', linewidth=1)  # 箱体边框颜色
        box.set(facecolor=colors[index], alpha=1)  # 箱体内部填充颜色
        box.set(hatch=hatchs[index])
        index += 1

    colors = ['#D5A3CB', '#F8B5C1', '#9FD18C', ]
    xlabels = ['', '              On Unseen QoE Preferences', '']
    hatchs = ['x', '//', '\\\\']
    boxes = plt.boxplot(data_2, notch=False, vert=True, patch_artist=True, labels=xlabels, showfliers=False,
                      showmeans=True, positions=positions + 5, widths=widths, meanprops=meanprops)

    index = 0
    for box in boxes['boxes']:
        box.set(color='black', linewidth=1)  # 箱体边框颜色
        box.set(facecolor=colors[index], alpha=1)  # 箱体内部填充颜色
        box.set(hatch=hatchs[index])
        index += 1

    # plt.ylim([0.4, 1.08])
    # plt.ylim([-0.7, 0.7])
    # plt.xticks([1, 2, 3, 4, 6, 7, 8, 9], labels)
    # plt.xticks([2.5,  7.5], labels)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.ylabel('Normalized Mean QoE', fontsize=14)
    # plt.title('Average QoE')
    plt.grid(axis='y', linestyle='--', alpha=0.5)
    # plt.grid(axis='x', linestyle='--', alpha=0.5)
    plt.tight_layout()
    labels = ['GEN360', 'GEN360 w/o RepL', 'GEN360 w/o BC']
    plt.legend(boxes['boxes'], labels, fontsize=13, ncol=4, loc='upper center')
    # plt.legend(boxes['boxes'], labels, fontsize=12, ncol=4, loc='upper center')
    # plt.savefig("overall_qoe_jin2022.pdf", dpi=300, format="pdf")
    plt.savefig("ablation_overall_qoe.pdf", dpi=300, format="pdf")
    plt.show()


plot_box_overall_qoe(gen360_seen, gen360_wo_repl_seen, gen360_wo_bc_seen, gen360_unseen, gen360_wo_repl_unseen, gen360_wo_bc_unseen)

In [ ]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt


def plot_box_overall_qoe(gen360_1, gen360_wo_repl_1, gen360_2, gen360_wo_repl_2):
    matplotlib.rcParams['hatch.linewidth'] = 1
    data_1 = []
    data_2 = []
    data_1.append(gen360_1[-1, -1, -1]['qoe'])
    data_1.append(gen360_wo_repl_1[-1, -1, -1]['qoe'])
    # print(rlva_1[-1, -1, -1]['qoe'])

    data_2.append(gen360_2[-1, -1, -1]['qoe'])
    data_2.append(gen360_wo_repl_2[-1, -1, -1]['qoe'])

    # search min, max for normalization
    min_1, max_1 = np.min(data_1), np.max(data_1)
    min_2, max_2 = np.min(data_2), np.max(data_2)

    # normalize
    # for i in range(len(data_1)):
    #     data_1[i] = (data_1[i] - min_1) / (max_1 - min_1)
    #     data_2[i] = (data_2[i] - min_2) / (max_2 - min_2)

    plt.figure(figsize=(7, 5))

    # colors = ['pink', 'lightblue', 'lightgreen', '#D38C80']
    colors = ['#D5A3CB', '#F8B5C1', ]
    xlabels = ['              On Trained QoE Preferences', '']
    hatchs = ['x', 'v',]
    widths = [0.6] * 2
    meanprops = dict(marker='^', markersize=8, markeredgecolor='black', markerfacecolor='orange', linewidth=0.0)
    positions = np.arange(1, 3)
    print(positions)

    boxes = plt.boxplot(data_1, notch=False, vert=True, patch_artist=True, showfliers=False, showmeans=True,
                        positions=positions, labels=xlabels, widths=widths, meanprops=meanprops)
    index = 0
    for box in boxes['boxes']:
        box.set(color='black', linewidth=1)  # 箱体边框颜色
        box.set(facecolor=colors[index], alpha=1)  # 箱体内部填充颜色
        box.set(hatch=hatchs[index])
        index += 1

    colors = ['#D5A3CB', '#F8B5C1', ]
    xlabels = ['              On Unseen QoE Preferences', '']
    hatchs = ['x', 'v',]
    boxes = plt.boxplot(data_2, notch=False, vert=True, patch_artist=True, labels=xlabels, showfliers=False,
                      showmeans=True, positions=positions + 5, widths=widths, meanprops=meanprops)

    index = 0
    for box in boxes['boxes']:
        box.set(color='black', linewidth=1)  # 箱体边框颜色
        box.set(facecolor=colors[index], alpha=1)  # 箱体内部填充颜色
        box.set(hatch=hatchs[index])
        index += 1

    # plt.ylim([0.4, 1.08])
    # plt.ylim([-0.7, 0.7])
    # plt.xticks([1, 2, 3, 4, 6, 7, 8, 9], labels)
    # plt.xticks([2.5,  7.5], labels)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.ylabel('Mean QoE', fontsize=14)
    # plt.title('Average QoE')
    plt.grid(axis='y', linestyle='--', alpha=0.5)
    # plt.grid(axis='x', linestyle='--', alpha=0.5)
    plt.tight_layout()
    labels = ['GEN360', 'GEN360 w/o RepL']
    plt.legend(boxes['boxes'], labels, fontsize=13, ncol=4, loc='upper center')
    # plt.legend(boxes['boxes'], labels, fontsize=12, ncol=4, loc='upper center')
    # plt.savefig("overall_qoe_jin2022.pdf", dpi=300, format="pdf")
    plt.savefig("ablation_overall_qoe.pdf", dpi=300, format="pdf")
    plt.show()


plot_box_overall_qoe(gen360_seen, gen360_wo_repl_seen, gen360_unseen, gen360_wo_repl_unseen)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

def plot_bar_individual_qoe(gen360, gen360_wo_repl, qoe_weight, title=None):
    data_qoe = []
    data_qoe1 = []
    data_qoe2 = []
    data_qoe3 = []

    data_qoe.append(gen360[qoe_weight]['qoe'])
    data_qoe.append(gen360_wo_repl[qoe_weight]['qoe'])

    data_qoe1.append(gen360[qoe_weight]['qoe1'])
    data_qoe1.append(gen360_wo_repl[qoe_weight]['qoe1'])

    data_qoe2.append(gen360[qoe_weight]['qoe2'])
    data_qoe2.append(gen360_wo_repl[qoe_weight]['qoe2'])

    data_qoe3.append(gen360[qoe_weight]['qoe3'])
    data_qoe3.append(gen360_wo_repl[qoe_weight]['qoe3'])

    # search min, max for normalization
    min_, max_ = np.min(data_qoe), np.max(data_qoe)
    min_1, max_1 = np.min(data_qoe1), np.max(data_qoe1)
    min_2, max_2 = np.min(data_qoe2), np.max(data_qoe2)
    min_3, max_3 = np.min(data_qoe3), np.max(data_qoe3)

    # normalize
    for i in range(len(data_qoe1)):
        data_qoe[i] = (data_qoe[i] - min_) / (max_ - min_)
        data_qoe1[i] = (data_qoe1[i] - min_1) / (max_1 - min_1)
        data_qoe2[i] = (data_qoe2[i] - min_2) / (max_2 - min_2)
        data_qoe3[i] = (data_qoe3[i] - min_3) / (max_3 - min_3)
    matplotlib.rcParams['hatch.linewidth'] = 2.5
    # fig, ax = plt.subplots(figsize=(8, 5))
    fig, ax = plt.subplots(figsize=(6, 3))
    width = 1
    qoe_num = 4
    method_num = 2
    x = []
    for i in range(qoe_num):
        for j in range(i + method_num * i, i + method_num * (i + 1)):
            x.append(j)
    print(x)
    total_data = []
    total_data_std = []
    tmp_data = [data_qoe, data_qoe1, data_qoe2, data_qoe3]
    for i in range(len(tmp_data)):
        for j in range(len(tmp_data[i])):
            total_data.append(np.mean(tmp_data[i][j]))
            total_data_std.append(np.std(tmp_data[i][j]))
            # total_data.append(np.mean(data_qoe1[i]))
            # total_data.append(np.mean(data_qoe2[i]))
            # total_data.append(np.mean(data_qoe3[i]))
    print(total_data)
    print(total_data_std)

    # data = [82.33797, 75.22432, 84.77188, 80.07726]
    colors = ['#6DBEC5', '#B9BB37'] * qoe_num
    # colors = ['#41644A', '#2A3990', '#41644A', '#2A3990']
    colors = ['#B0D9E7', '#D5A3CB'] * qoe_num
    hatchs = ['++', 'xxx'] * qoe_num
    # hatchs = ['////', '\\\\\\\\', '+++', 'xxx'] * method_num
    labels = ['GEN360', 'GEN360 w/o RepL' '', '', '', '', '', '', '']
    print(len(x), len(colors), len(hatchs), len(labels))
    for i in range(len(x)):
        ax.bar(x[i] - 0.5 * width, total_data[i], width, color='white', edgecolor=colors[i],
                        hatch=hatchs[i], zorder=100, lw=1., label=labels[i])
        # ax.bar(x[i] - 0.5 * width, total_data[i], width, color=colors[i], zorder=100, lw=1., edgecolor='black', label=labels[i])
        # ax.bar(x[i] - 0.5 * width, total_data[i], width, color='none', edgecolor='white',
        #                 hatch=hatchs[i], zorder=200, lw=1.)
        ax.bar(x[i] - 0.5 * width, total_data[i], width, color='none', edgecolor='black', zorder=200, lw=1., )
        # ax.errorbar(x[i] - 0.5 * width, total_data[i], yerr=total_data_std[i], color='black', lw=1., capsize=4, zorder=200)
        # ax.text(x[i] - 0.85 * width, data[i] + 1, '%.2f' % data[i], fontsize=12)
        # ax.text(x[i] - 0.85 * width, total_data[i] + 1, '%.1f' % total_data[i], fontsize=12)
    # xlabels = ['', '          On Trained Viewing Patterns', '', '', '',
    #            '', '            On Unseen Viewing Patterns', '', '', '']
    xlabels = ['              Overall QoE$\\uparrow$', '', 
               '             Viewport\n              Quality$\\uparrow$', '',  
               '              Rebuffering$\\downarrow$', '', 
               '            Smoothness\n             Penalty$\\downarrow$', '', ]
    # xlabels = ['        Overall QoE', '', '', '',
    #            '       Viewport Quality', '', '', '', 
    #            '       Rebuffering Time', '', '', '', 
    #            '       Smoothness Penalty', '', '', '',]
    ax.set_xticks(x)
    ax.set_xticklabels(xlabels, rotation=5)  # 14
    ax.set_ylim([0, 1])
    # ax.set_yticks([14, 15, 16, 17, 18])
    ax.set_ylabel('Normalized Mean Scores', fontsize=12)
    # ax.set_xlabel(' ', fontsize=14)
    # ax.legend(loc='upper right', fontsize=14, ncol=2, borderaxespad=0)  # 14
    ax.legend(loc='upper right', fontsize=10, ncol=2, borderaxespad=0)  # 14
    ax.grid(axis='y', alpha=0.6, zorder=10)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    if title is None:
        title = f'QoE Preference {qoe_weight}'
    plt.title(title, fontsize=12)
    plt.tight_layout()
    save_dir = ''
    # plt.savefig(f"individual_qoe_jin2022_{'_'.join(map(str, qoe_weight))}.pdf", dpi=300, format="pdf")
    plt.savefig(f"ablation_individual_qoe_{'_'.join(map(str, qoe_weight))}.pdf", dpi=300, format="pdf")
    # save_path = save_dir + 'motivation_accuracy_seen_unseen.pdf'
    # if save_path is not None:
    #     plt.savefig(save_path, dpi=600, bbox_inches='tight')
    plt.show()
    
plot_bar_individual_qoe(gen360_seen, gen360_wo_repl_seen, (7, 1, 1), title='QoE Preference ($\\frac{7}{9}$, $\\frac{1}{9}$, $\\frac{1}{9}$)')
plot_bar_individual_qoe(gen360_seen, gen360_wo_repl_seen, (1, 7, 1), title='QoE Preference ($\\frac{1}{9}$, $\\frac{7}{9}$, $\\frac{1}{9}$)')
# plot_bar_individual_qoe(gen360_seen, gen360_wo_repl_seen, (1, 1, 7), title='QoE Preference ($\\frac{1}{9}$, $\\frac{1}{9}$, $\\frac{7}{9}$)')
# plot_bar_individual_qoe(gen360_seen, gen360_wo_repl_seen, (3, 3, 3), title='QoE Preference ($\\frac{3}{9}$, $\\frac{3}{9}$, $\\frac{3}{9}$)')

plot_bar_individual_qoe(gen360_unseen, gen360_wo_repl_unseen, (5, 1, 3), title='QoE Preference ($\\frac{5}{9}$, $\\frac{1}{9}$, $\\frac{3}{9}$)')
# plot_bar_individual_qoe(gen360_unseen, gen360_wo_repl_unseen, (2, 4, 3), title='QoE Preference ($\\frac{2}{9}$, $\\frac{4}{9}$, $\\frac{3}{9}$)')
plot_bar_individual_qoe(gen360_unseen, gen360_wo_repl_unseen, (1, 3, 5), title='QoE Preference ($\\frac{1}{9}$, $\\frac{3}{9}$, $\\frac{5}{9}$)')
plot_bar_individual_qoe(gen360_unseen, gen360_wo_repl_unseen, (4, 4, 1), title='QoE Preference ($\\frac{4}{9}$, $\\frac{4}{9}$, $\\frac{1}{9}$)')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


def read_file(filename):
    data = [[0, -1089.3643798828125]]
    with open(filename, 'r') as file:
        file.readline()
        for line in file.readlines():
            line = line.strip().split(',')
            data.append([int(line[-2]), float(line[-1])])
    return np.array(data)


with_bc_path = 'bc_reward_csv.csv'
with_bc_std_path = 'bc_reward_std_csv.csv'
without_bc_path = 'no_bc_reward_csv.csv'
without_bc_std_path = 'no_bc_reward_std_csv.csv'
with_bc = read_file(with_bc_path)
with_bc_std = read_file(with_bc_std_path)
without_bc = read_file(without_bc_path)
without_bc_std = read_file(without_bc_std_path)

epoch_num = min(len(with_bc), len(without_bc))
epoch_num = 200
plt.plot(range(epoch_num), with_bc[:epoch_num, 1], label='GEN360', color='red')
plt.plot(range(epoch_num), without_bc[:epoch_num, 1], label='GEN360 w/o BC Init', color='blue')
plt.xlabel('Epoch')
plt.ylabel('QoE Scores')
# plt.plot(with_bc[:epoch_num, 0], with_bc[:epoch_num, 1], label='GEN360', color='red')
# plt.plot(without_bc[:epoch_num, 0], without_bc[:epoch_num, 1], label='GEN360 w/o BC Init', color='blue')
# plt.xlabel('Steps')
plt.legend()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


def read_file2(filename):
    data = []
    with open(filename, 'r') as file:
        file.readline()
        for line in file.readlines():
            line = line.strip().split(',')
            data.append([float(line[0]), float(line[1]), float(line[2]), float(line[3])])
    return np.array(data)


with_id_path = '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/gen360_3/Wu2017_4G/unseen_qoe3/epochs_20_bs_256_lr_0.0001_gamma_0.99_seed_1_ent_0.01_useid_True_lambda_0.5_ilr_0.0001_iur_2_bc_True/results_case_study_uid_True.csv'
without_id_path = '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/gen360_3/Wu2017_4G/unseen_qoe3/epochs_20_bs_256_lr_0.0001_gamma_0.99_seed_1_ent_0.01_useid_False_lambda_0.5_ilr_0.0001_iur_2_bc_True/results_case_study_uid_False.csv'
with_id = read_file2(with_id_path)
without_id = read_file2(without_id_path)
length = len(with_id)
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()  # Create a twin Axes sharing the x-axis
ax1.plot(range(length), with_id[:, -2], 'r-', label='GEN360')  
ax1.plot(range(length), without_id[:, -2], 'g-', label='GEN360 w/o RepL')  
ax2.plot(range(length), with_id[:, -1], 'r.', label='GEN360')  
ax2.plot(range(length), without_id[:, -1], 'g.', label='GEN360 w/o RepL')  
ax1.set_xlabel('Chunk ID')
ax1.set_ylabel('Rebuffering Time', color='g')
ax2.set_ylabel('Quality', color='b')
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


def moving_average(data, window_size=10):
    window = np.ones(window_size) / window_size
    smoothed_data = np.convolve(data, window, mode='same')
    return smoothed_data

def read_file2(filename):
    data = []
    with open(filename, 'r') as file:
        file.readline()
        for line in file.readlines():
            line = line.strip().split(',')
            data.append([float(line[0]), float(line[1]), float(line[2]), float(line[3])])
    return np.array(data)


# with_id_path = '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/gen360_3/Wu2017_4G/unseen_qoe3/epochs_20_bs_256_lr_0.0001_gamma_0.99_seed_1_ent_0.01_useid_True_lambda_0.5_ilr_0.0001_iur_2_bc_True/results_case_study_uid_True.csv'
# without_id_path = '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/gen360_3/Wu2017_4G/unseen_qoe3/epochs_20_bs_256_lr_0.0001_gamma_0.99_seed_1_ent_0.01_useid_False_lambda_0.5_ilr_0.0001_iur_2_bc_True/results_case_study_uid_False.csv'
# with_id_path = '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/gen360_3/Wu2017_4G/case_study/epochs_2000_bs_512_lr_0.0001_gamma_0.99_seed_1_ent_0.01_useid_True_lambda_0.5_ilr_0.0001_iur_2_bc_False/results_case_study_uid_True_v_2_u_25_t27_fqoe_1_7_1_sqoe_7_1_1.csv'
# without_id_path = '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/gen360_3/Wu2017_4G/case_study/epochs_2000_bs_512_lr_0.0001_gamma_0.99_seed_1_ent_0.01_useid_False_lambda_0.5_ilr_0.0001_iur_2_bc_False/results_case_study_uid_False_v_2_u_25_t27_fqoe_1_7_1_sqoe_7_1_1.csv'
# with_id_path = '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/gen360_3/Wu2017_4G/case_study/epochs_2000_bs_512_lr_0.0001_gamma_0.99_seed_1_ent_0.01_useid_True_lambda_0.5_ilr_0.0001_iur_2_bc_False/results_case_study_uid_True_v_2_u_25_t27_fqoe_7_1_1_sqoe_1_7_1.csv'
# without_id_path = '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/gen360_3/Wu2017_4G/case_study/epochs_2000_bs_512_lr_0.0001_gamma_0.99_seed_1_ent_0.01_useid_False_lambda_0.5_ilr_0.0001_iur_2_bc_False/results_case_study_uid_False_v_2_u_25_t27_fqoe_7_1_1_sqoe_1_7_1.csv'
# with_id_path =  '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/gen360_3/Wu2017_4G/case_study/epochs_2000_bs_512_lr_0.0001_gamma_0.99_seed_1_ent_0.01_useid_True_lambda_0.5_ilr_0.0001_iur_2_bc_False/results_case_study_uid_True_v_2_u_25_t16_fqoe_1_7_1_sqoe_7_1_1.csv'
# without_id_path = '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/gen360_3/Wu2017_4G/case_study/epochs_2000_bs_512_lr_0.0001_gamma_0.99_seed_1_ent_0.01_useid_False_lambda_0.5_ilr_0.0001_iur_2_bc_False/results_case_study_uid_False_v_2_u_25_t16_fqoe_1_7_1_sqoe_7_1_1.csv'
# with_id_path =  '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/gen360_3/Wu2017_4G/case_study/epochs_2000_bs_512_lr_0.0001_gamma_0.99_seed_1_ent_0.01_useid_True_lambda_0.5_ilr_0.0001_iur_2_bc_False/results_case_study_uid_True_v_2_u_17_t16_fqoe_1_7_1_sqoe_7_1_1.csv'
# without_id_path = '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/gen360_3/Wu2017_4G/case_study/epochs_2000_bs_512_lr_0.0001_gamma_0.99_seed_1_ent_0.01_useid_False_lambda_0.5_ilr_0.0001_iur_2_bc_False/results_case_study_uid_False_v_2_u_17_t16_fqoe_1_7_1_sqoe_7_1_1.csv'
with_id_path =  '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/gen360_3/Wu2017_4G/case_study/epochs_2000_bs_512_lr_0.0001_gamma_0.99_seed_1_ent_0.01_useid_True_lambda_0.5_ilr_0.0001_iur_2_bc_False/results_case_study_uid_True_v_2_u_17_t23_fqoe_1_7_1_sqoe_7_1_1.csv'
without_id_path = '/data/wuduo/2023_omnidirectional_vs/results/bitrate_selection/gen360_3/Wu2017_4G/case_study/epochs_2000_bs_512_lr_0.0001_gamma_0.99_seed_1_ent_0.01_useid_False_lambda_0.5_ilr_0.0001_iur_2_bc_False/results_case_study_uid_False_v_2_u_17_t23_fqoe_1_7_1_sqoe_7_1_1.csv'



with_id = read_file2(with_id_path)
without_id = read_file2(without_id_path)
length = len(with_id)
fig, ax1 = plt.subplots(figsize=(6, 2))
# ax1.plot(range(length), with_id[:, -2], 'r-', label='GEN360')  
# ax1.plot(range(length), without_id[:, -2], 'g-', label='GEN360 w/o RepL')  
ax1.plot(range(length), moving_average(with_id[:, -2]), 'r-', label='GEN360')  
ax1.plot(range(length), moving_average(without_id[:, -2]), 'g-', label='GEN360 w/o RepL')  
ax1.set_xlabel('Chunk ID')
ax1.set_ylabel('Rebuffering Time')
ax1.legend(loc='upper left')
plt.show()
fig, ax2 = plt.subplots(figsize=(6, 2))
ax2.plot(range(length), moving_average(with_id[:, -3]), 'r-', label='GEN360')  
ax2.plot(range(length), moving_average(without_id[:, -3]), 'g-', label='GEN360 w/o RepL')  
ax2.set_xlabel('Chunk ID')
ax2.set_ylabel('Quality')
ax2.legend(loc='upper left')
plt.show()
fig, ax2 = plt.subplots(figsize=(6, 2))
ax2.plot(range(length), moving_average(with_id[:, 0]), 'r-', label='GEN360')  
ax2.plot(range(length), moving_average(without_id[:, 0]), 'g-', label='GEN360 w/o RepL')  
ax2.set_xlabel('Chunk ID')
ax2.set_ylabel('Quality')
ax2.legend(loc='upper left')
plt.show()